* Check the python version by {python --version} make sure its higher or equal to 3.9

In [6]:
# pip install selenium beautifulsoup4 pandas webdriver-manager on terminal

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import math
import re
import os

In [8]:
# ---------------- SETUP ----------------
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options) # If this does not work, uncomment the below code and comment this line and comment this line 
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(
#     service=Service(ChromeDriverManager().install()),
#     options=options
# )

In [9]:
# ---------------- LIST OF URLS ----------------
URLS = [ # Add your URLs here
    "https://www.tripadvisor.com/Attraction_Review-g1407334-d6104104-Reviews-Mirissa_Beach-Mirissa_Southern_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304134-d7034305-Reviews-Hikkaduwa_Beach-Hikkaduwa_Galle_District_Southern_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g644047-d3331573-Reviews-Jungle_Beach-Unawatuna_Galle_District_Southern_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g297897-d1655656-Reviews-Negombo_Beach-Negombo_Western_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g293962-d554029-Reviews-Mount_Lavinia_Beach-Colombo_Western_Province.html",
]

In [10]:
MAX_REVIEWS_PER_DESTINATION = 500 # Manjitha should change this value for 500

# ---------------- OUTPUT FOLDER ----------------
output_folder = r"C:\NIBM\Travel recommendation system\Travel-Recommendation-system-in-Sri-Lanka-\scraped_data" #your file path
os.makedirs(output_folder, exist_ok=True)

# ---------------- SCRAPING ----------------
for full_url in URLS:
    print(f"\nProcessing destination: {full_url}")
    driver.get(full_url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")

  # ---------------- DESTINATION TITLE ----------------
    title_tag = soup.select_one("h1[data-test-target='mainH1']")
    destination_title = title_tag.get_text(strip=True) if title_tag else "Unknown"

    safe_title = re.sub(r'[\\/*?:"<>|]', "", destination_title)
    filename = os.path.join(output_folder, f"{safe_title}.csv")

    # ---------------- SKIP IF FILE EXISTS ----------------
    if os.path.exists(filename):
        print(f"CSV already exists for {destination_title}, skipping...")
        continue

    # ---------------- DESTINATION SCORE ----------------
    score_tag = soup.select_one("div[data-automation='bubbleRatingValue'] span")
    destination_score = score_tag.get_text(strip=True) if score_tag else None

    # ---------------- TOTAL REVIEWS ----------------
    total_tag = soup.select_one("div[data-automation='bubbleReviewCount'] span")
    total_reviews = 0
    if total_tag:
        total_reviews = int(re.sub(r"[^\d]", "", total_tag.get_text()))
    total_reviews = min(total_reviews, MAX_REVIEWS_PER_DESTINATION)

    # ---------------- DESTINATION IMAGE ----------------
    img_tag = soup.select_one("img[srcset]")
    destination_image = img_tag["src"] if img_tag else None

    print(f"Destination: {destination_title} | Reviews to scrape: {total_reviews}")

    # ---------------- REVIEW LEVEL ----------------
    reviews_per_page = 10
    total_pages = math.ceil(total_reviews / reviews_per_page)
    all_reviews = []

    for page in range(total_pages):
        if page == 0:
            page_url = full_url
        else:
            page_url = full_url.replace("Reviews-", f"Reviews-or{page * reviews_per_page}-")

        print(f"  Scraping page {page + 1}/{total_pages}")
        driver.get(page_url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        review_cards = soup.select("div[data-automation='reviewCard']")

        for card in review_cards:

            # Reviewer name
            user_tag = card.select_one("span.biGQs._P.ezezH a")
            user = user_tag.get_text(strip=True) if user_tag else None

            # Reviewer location
            location_tag = card.select_one("div.biGQs._P.navcl span")
            location = location_tag.get_text(strip=True) if location_tag else None

            # Review title
            title_tag = card.select_one("h3 span.yCeTE")
            review_title = title_tag.get_text(strip=True) if title_tag else None

            # Review text
            review_tag = card.select_one("div.biGQs._P.VImYz.AWdfh span.yCeTE")
            review_text = review_tag.get_text(" ", strip=True) if review_tag else None

            # Written date
            written_tag = card.select_one("div.biGQs._P.VImYz.ncFvv.navcl")
            written_date = (
                written_tag.get_text(strip=True).replace("Written ", "")
                if written_tag else None
            )

            # Review score
            review_score = None
            score_svg = card.select_one("svg[data-automation='bubbleRatingImage'] title")
            if score_svg:
                match = re.search(r"(\d) of 5", score_svg.get_text())
                if match:
                    review_score = int(match.group(1))

            # Reviewer category (ROBUST METHOD)
            category = None
            card_text = card.get_text(" ", strip=True)
            cat_match = re.search(
                r"\b(Couples|Family|Friends|Solo|Business)\b",
                card_text
            )
            if cat_match:
                category = cat_match.group(1)

            all_reviews.append({
                "Destination URL": full_url,
                "Destination Title": destination_title,
                "Destination Score": destination_score,
                "Total Reviews (Destination)": total_reviews,
                "Destination Image": destination_image,
                "Reviewer Name": user,
                "Reviewer Location": location,
                "Review Title": review_title,
                "Review Score": review_score,
                "Reviewer Category": category,
                "Review Text": review_text,
                "Written Date": written_date
            })

    # ---------------- SAVE CSV ----------------
    df = pd.DataFrame(all_reviews)
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"Saved {len(df)} reviews → {filename}")

driver.quit()
print("\n✅ Scraping completed successfully.")


Processing destination: https://www.tripadvisor.com/Attraction_Review-g1407334-d6104104-Reviews-Mirissa_Beach-Mirissa_Southern_Province.html
Destination: Mirissa Beach | Reviews to scrape: 500
  Scraping page 1/50


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=143.0.7499.40); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff681eb88e5
	0x7ff681eb8940
	0x7ff681c9165d
	0x7ff681c7d202
	0x7ff681ca27af
	0x7ff681d19a29
	0x7ff681d3a5c2
	0x7ff681cdac29
	0x7ff681cdba93
	0x7ff6821d0640
	0x7ff6821caf80
	0x7ff6821e96e6
	0x7ff681ed5de4
	0x7ff681eded8c
	0x7ff681ec2004
	0x7ff681ec21b5
	0x7ff681ea7ee2
	0x7ffbf303e8d7
	0x7ffbf4e0c53c
